<a href="https://colab.research.google.com/github/ingabLee/Transformers_book/blob/main/TransformerLearning_Chap91_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wikipedia

import wikipedia

keyword = "Moderna"

#set lang en
wikipedia.set_lang("en")

# search keyword and display output
search_response = wikipedia.search(keyword)
print(search_response)

In [ ]:
search_response[2]

# search_response의 세번째 요소를 키워드로 위키피디아 검색
moderna_text_t = wikipedia.page(search_response[2])
type(moderna_text_t)

# search_response 세번째 요소를 키워드로 위키피디아 검색결과 본문 출력
moderna_text = wikipedia.page(search_response[2]).content
moderna_text

In [ ]:
keyword = "PFizer"

wikipedia.set_lang("en")
search_response = wikipedia.search(keyword)
print(search_response)

pfizer_text = wikipedia.page(search_response[1]).content

In [ ]:
from transformers import pipeline, AutoTokenizer

# 파이프라인 인스턴스화
get_feature = pipeline('feature-extraction', model='bert-base-uncased', tokenizer='bert-base-uncased')

In [ ]:
sample_word = "vaccine"

# sample word를 파이프라인을 인스턴스화한 get_feature에 투입
hidden_state = get_feature(sample_word)

import numpy as np
np.array(hidden_state).shape

In [ ]:
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# sample word를 토큰나이징한 결과의 input_ids를 디코딩
print(tokenizer.decode(tokenizer(sample_word)["input_ids"]))

In [ ]:
# get_cls_vector 함수 정의
def get_cls_vector(sample_text):
  hidden_state = get_feature(sample_text, padding=True, truncation=True, max_length=512)

  # hidden_state를 넘파이 얼레이 타입으로 바꾸고 첫번째 행의 첫번째 열 요소를 변수 cls_vec에 저장
  cls_vec = np.array(hidden_state)[0, 0]
  return cls_vec

# 입력 텍스트 줄이 바뀔때마다 분리후 저장
ml = moderna_text.split("\n")

pfzl = pfizer_text.split("\n")

# 입력 텍스트중에 공백은 제거
ml = [text for text in ml if text != ""]
pfzl= [text for text in pfzl if text != ""]


In [ ]:
# runtime 50sec
# ml 및 pfzl에 담긴 텍스트를 get_cls_vector함수에 입력후 그 결과를 넘파이 어레이로 변환
moderna_vecs = np.array([get_cls_vector(text) for text in ml])
pfizer_vecs = np.array([get_cls_vector(text) for text in pfzl])

# check moderna_vecs, pfizer_vecs차원 확인
print(moderna_vecs.shape)
print(pfizer_vecs.shape)

In [ ]:
# empty dictionary
pfizer_dics = {}

# for loop for pfizer
for text in pfzl:
  # text tokenizing result for loop
  for token in tokenizer.tokenize(text):

    # token length > 5
    if len(token) > 5:
      # pfizer_dic not exist key
      if token not in pfizer_dics:
        #add new  dictionary item
        pfizer_dics[token] = 1

      # exist key, add 1
      else:
        pfizer_dics[token] += 1

# pfizer_dics.items()는 pfizer_dics 딕셔너리 키와 값 출력
# sorted(pfizer_dic.items(), key=lambda x:-x[1])
# dictionary key value기준으로 내림차순 정렬
# [:30] 결과물의 첫 30개 출력
sorted(pfizer_dics.items(), key=lambda x:-x[1])[:30]

In [ ]:
ml_dics = {}

for text in ml:
  for token in tokenizer.tokenize(text):
    if len(token) > 5:
      if token not in ml_dics:
        ml_dics[token] = 1
      else:
        ml_dics[token] += 1

sorted(ml_dics.items(), key=lambda x:-x[1])[:30]

In [ ]:
# TSNE import
from sklearn.manifold import TSNE

# TSNE parameter setting
tsne = TSNE(n_components=2, random_state=0)

# TSNE에 moderna_vecs투입후 적용(fit_transfrom)
moderna_vecs_reduced = tsne.fit_transform(moderna_vecs)

# plot display
import matplotlib.pyplot as plt
plt.figure(figsize=(13, 7))

# moderna_vecs_reduced[:, 0] -> 첫번째 컬럼값의 모든 행 데이터 의미
# moderna_vecs_reduced[:, 1] -> 두번째 컬럼값의 모든 행 데이터 의미
# c는 작은원으로 찍히는 데이터의 색상의 종류로 len(ml)개수 만큼 설정
plt.scatter(moderna_vecs_reduced[:, 0], moderna_vecs_reduced[:, 1], c=[i for i in range(len(ml))])

#graph right color bar
plt.colorbar()
plt.show();

In [ ]:
tsne = TSNE(n_components=2, random_state=0)
pfizer_vecs_reduced = tsne.fit_transform(pfizer_vecs)

plt.figure(figsize=(13, 7))
plt.scatter(pfizer_vecs_reduced[:, 0], pfizer_vecs_reduced[:, 1],
            c=[i for i in range(len(pfzl))])
plt.colorbar()
plt.show();